In [1]:
# definition for a binary tree node.
from typing import Optional, List

class TreeNode:
	def __init__(self, val=0, left=None, right=None):
		self.val = val
		self.left = left
		self.right = right

# construct binary tree from list.
def constructBinaryTree(nodes: List[int]) -> TreeNode:
	# if the list is empty, return None.
	if not nodes: return None

	root = TreeNode(nodes[0], None, None)
	queue = [root]

	for i, v in enumerate(nodes[1:]):
		# if v is None, the node is None.
		node = TreeNode(v, None, None) if v is not None else None

		# skip None node, it only a placehoder.
		while queue[0] == None:
			queue.pop(0)
		
		if not i % 2:
			queue[0].left = node
		else:
			queue[0].right = node
			queue.pop(0)
		
		queue.append(node)
	
	return root

def convertBinaryTree2List(root: TreeNode) -> List[int]:
	if not root: return []

	queue = [root]
	ans = []

	while queue:
		n = len(queue)
		for _ in range(n):
			node = queue.pop(0)
			if node:
				queue.append(node.left)
				queue.append(node.right)
				ans.append(node.val)
			else:
				ans.append(node)
	
	for i in range(len(ans)-1, -1, -1):
		if ans[i]: return ans[:i+1]

root = [3,9,20,None,None,15,7]
root_node = constructBinaryTree(root)
tree_list = convertBinaryTree2List(root_node)
tree_list

[3, 9, 20, None, None, 15, 7]

In [3]:
class Solution:
	# Breadth-First Search, also called Level-Order Traversal.
	def levelOrderTraversal(self, root: Optional[TreeNode]) -> List[List[int]]:
		# if the tree is empty, return empty list.
		if not root: return []

		queue = [root]
		ans = []

		while queue:
			# record the node number of the current level.
			n = len(queue)
			cur_level_vals = []
			# traverse the current level nodes.
			for _ in range(n):
				node = queue.pop(0)
				if node.left: queue.append(node.left)
				if node.right: queue.append(node.right)
				cur_level_vals.append(node.val)
			ans.append(cur_level_vals)
		
		return ans
	
	# Depth-First Search, including Pre-Order, In-Order and Post-Order Traversal.
	def preOrderTraversal(self, root: Optional[TreeNode]) -> List[int]:
		# if the tree is empty, return empty list.
		if not root: return []

		ans = [root.val]
		ans += self.preOrderTraversal(root.left)
		ans += self.preOrderTraversal(root.right)

		return ans
	
	def inOrderTraversal(self, root: Optional[TreeNode]) -> List[int]:
		# if the tree is empty, return empty list.
		if not root: return []
		
		ans = []
		ans += self.inOrderTraversal(root.left)
		ans += [root.val]
		ans += self.inOrderTraversal(root.right)

		return ans

	def postOrderTraversal(self, root: Optional[TreeNode]) -> List[int]:
		# if the tree is empty, return empty list.
		if not root: return []

		ans = []
		ans += self.postOrderTraversal(root.left)
		ans += self.postOrderTraversal(root.right)
		ans += [root.val]

		return ans

root = [3,9,20,None,None,15,7]
root = constructBinaryTree(root)

solution = Solution()
solution.levelOrderTraversal(root)
# [[3], [9, 20], [15, 7]]
solution.preOrderTraversal(root)
# [3, 9, 20, 15, 7]
solution.inOrderTraversal(root)
# [9, 3, 15, 20, 7]
solution.postOrderTraversal(root)
# [9, 15, 7, 20, 3]

[9, 15, 7, 20, 3]

In [4]:
class Solution:
	def numTrees(self, n: int) -> int:
		dp = [0] * (n+1)
		# boundary conditions.
		dp[0] = 1
		dp[1] = 1

		for i in range(2, n+1):
			for j in range(i):
				dp[i] += dp[j] * dp[i-j-1]
		
		return dp[n]

	def generateTrees(self, n: int) -> List[Optional[TreeNode]]:
		if not n: return []

		hashtable = dict()


		def _generateTrees(start, end):
			if start > end: return [None]
			if (start, end) in hashtable:
				return hashtable[(start, end)]
			
			ans = []
			
			for i in range(start, end+1):
				left_subtrees = _generateTrees(start, i-1)
				right_subtrees = _generateTrees(i+1, end)
				for left_root in left_subtrees:
					for right_root in right_subtrees:
						root = TreeNode(i, left_root, right_root)
						ans.append(root)
			hashtable[(start, end)] = ans

			return ans


		return _generateTrees(1, n)
	
	def isValidBST(self, root: Optional[TreeNode]) -> bool:
		if not root: return False

		def _checkBST(node, lower, upper):
			# leaf node
			if not node: return True
			
			if node.val <= lower or node.val >= upper: return False

			return _checkBST(node.left, lower, node.val) and _checkBST(node.right, node.val, upper)

		return _checkBST(root.left, -2**32-1, root.val) and _checkBST(root.right, root.val, 2**31)
	
	def isValidBST(self, root: Optional[TreeNode]) -> bool:
		if not root: return False

		pre_val = root.val
		
		def _inOrderTraversal(root: Optional[TreeNode]):
			if not root: return True

			if not _inOrderTraversal(root.left): return False

			if root.val <= pre_val:
				return False
			
			return _inOrderTraversal(root.right)
		return _inOrderTraversal(root)
	
	def isValidBST3(self, root: Optional[TreeNode]) -> bool:
		if not root: return False

		# -inf boundary -2**31-1
		lower_node = TreeNode(-2**31-1, None, None)
		upper_node = TreeNode(2**31, None, None)
		stack = [lower_node, root, upper_node]

		while stack:
			upper_node = stack.pop()
			node = stack.pop()

			while not node.left and not node.right and stack:
				upper_node = node
				node = stack.pop()
			
			if not stack: return True

			if node.left:
				if (node.left.val >= node.val or node.left.val <= stack[-1].val):
					return False
				else:
					stack.append(node.left)
					node.left = None
			
			right_node, node.right = node.right, None
			
			stack.append(node)
			
			if right_node:
				if (right_node.val <= node.val or right_node.val >= upper_node.val):
					return False
				else:
					stack.append(right_node)
			stack.append(upper_node)
	
	def recoverTree(self, root: Optional[TreeNode]) -> None:
		if not root: return root

		self.pre_node = TreeNode(-2**31-1)
		self.swap_node_x, self.swap_node_y = None, None

		def _inOrderTraversal(root):
			if not root:
				print(root)
				return;

			_inOrderTraversal(root.left)

			if root.val <= self.pre_node.val:
				if not self.swap_node_x: self.swap_node_x = self.pre_node
				self.swap_node_y = root
		
			self.pre_node = root

			_inOrderTraversal(root.right)

		_inOrderTraversal(root)

		if self.swap_node_x and self.swap_node_y:
			self.swap_node_x.val, self.swap_node_y.val = self.swap_node_y.val, self.swap_node_x.val


n = 3
solution = Solution()
# solution.numTrees(n)
# ans = solution.generateTrees(n)
# for root in ans:
# 	print(convertBinaryTree2List(root))
root = [5,1,4,None,None,3,6]
root = [2,1,3]
root = [32,26,47,19,None,None,56,None,27]
# root = [5,4,6,None,None,3,7]
# root = [0, None, -1]


root = [1,3,None,None,2]
root = constructBinaryTree(root)
# solution.isValidBST2(root)
solution.recoverTree(root)
convertBinaryTree2List(root)

None
None
None
None


[3, 1, None, None, 2]

In [ ]:
class Solution:
	def isSameTree(self, p: Optional[TreeNode], q: Optional[TreeNode]) -> bool:
		if not p and not q: return True
		if not p or not q: return False

		if p.val != q.val:
			return False
			
		return self.isSameTree(p.left, q.left) and self.isSameTree(p.right, q.right)
	
	# recursive algorithm.
	def isSymmetric(self, root: Optional[TreeNode]) -> bool:
		if not root: return False

		def _isSymmetric(p: Optional[TreeNode], q: Optional[TreeNode]) -> bool:
			if not p and not q: return True
			if not p or not q: return False

			if p.val != q.val:
				return False
			return _isSymmetric(p.left, q.right) and _isSymmetric(p.right, q.left)

		return _isSymmetric(root.left, root.right)
	
	# level traversal.
	def isSymmetric_stack(self, root: Optional[TreeNode]) -> bool:
		if not root: return False

		queue = [root]

		while queue:
			n = len(queue)
			level_vals = []

			for _ in range(n):
				node = queue.pop(0)
				if node: 
					queue.append(node.left)
					queue.append(node.right)
					level_vals.append(node.val) 
				else: 
					level_vals.append(node)
			
			for i in range(n // 2 + 1):
				if level_vals[i] != level_vals[n-i-1]:
					return False

		return True


p = [1 ,2]
q = [1,None,2]

p = [1,2,3]
q = [1,2,3]

p = [1,2,1]
q = [1,1,2]
p, q = constructBinaryTree(p), constructBinaryTree(q)

solution = Solution()
ans = solution.isSameTree(p, q)
ans

root = [1,2,2,None,3,None,3]
# root = [1,2,2,3,4,4,3]
root = constructBinaryTree(root)
solution = Solution()
ans = solution.isSymmetric_stack(root)
ans

False

In [16]:
class Solution:
	def zigzagLevelOrder(self, root: Optional[TreeNode]) -> List[List[int]]:
		if not root: return []

		ans = []
		queue = [root]
		order = True

		while queue:
			n = len(queue)
			level_vals = []

			for _ in range(n):
				node = queue.pop(0)
				level_vals.append(node.val)
				
				if node.left: queue.append(node.left)
				if node.right: queue.append(node.right)
			if order:
				ans.append(level_vals)
			else:
				level_vals.reverse()
				ans.append(level_vals)
		
			order = not order
			
		return ans
	

root = [3,9,20,None,None,15,7]
root = constructBinaryTree(root)
solution = Solution()
solution.zigzagLevelOrder(root)

[[3], [20, 9], [15, 7]]

In [ ]:
# The max and min depth of a binary tree
class Solution:
	def maxDepth(self, root: Optional[TreeNode]) -> int:
		if not root: return 0

		depth = 0
		queue = [root]

		while queue:
			depth += 1
			
			for _ in range(len(queue)):
				node = queue.pop(0)
				if node.left: queue.append(node.left)
				if node.right: queue.append(node.right)
		
		return depth

	def minDepth(self, root: Optional[TreeNode]) -> int:
		if not root: return 0

		depth = 0
		queue = [root]

		while queue:
			depth += 1
			for _ in range(len(queue)):
				node = queue.pop(0)
				if not node.left and not node.right: return depth
				if node.left: queue.append(node.left)
				if node.right: queue.append(node.right)
        

root = [3,9,20,None,None,15,7]
root = constructBinaryTree(root)
solution = Solution()
solution.maxDepth(root)
solution.minDepth(root)

2

In [ ]:
# Balanced (AVL) Tree
class Solution:
	def isBalanced(self, root: Optional[TreeNode]) -> bool:
		"""Pre-order Traversal (from top to bottom)
		"""
		if not root: return True

		def depth(root: Optional[TreeNode]) -> int:
			if not root: return 0
			return max(depth(root.left), depth(root.right)) + 1
		
		return abs(depth(root.left) - depth(root.right)) <= 1 and \
			self.isBalanced(root.left) and self.isBalanced(root.right)
	

	def isBalanced(self, root: Optional[TreeNode]) -> bool:
		"""Post-order Traversal"""
		def depth(root: Optional[TreeNode]) -> int:
			if not root: return 0

			left_depth = depth(root.left)
			if left_depth == -1: return -1
			right_depth = depth(root.right)
			if right_depth == -1: return -1

			return max(left_depth, right_depth) + 1 \
				if abs(left_depth - right_depth) <= 1 else -1
		
		return depth(root) != -1


# True
root = [3,9,20,None,None,15,7]
# False
root = [1,2,2,3,3,None,None,4,4]
# True
# root = []

root = constructBinaryTree(root)

solution = Solution()
solution.isBalanced(root)

False

In [2]:
# Sum Path
class Solution:
	def hasPathSum(self, root: Optional[TreeNode], targetSum: int) -> bool:
		"""Post-order Traversal"""
		if not root: return False

		stack = [(root, root.val)]

		while stack:
			node, path = stack.pop()
			if not node.left and not node.right and path == targetSum:
				return True
			if node.left:
				stack.append((node.left, path + node.left.val))
			if node.right:
				stack.append((node.right, path + node.right.val))

		return False

	def pathSum(self, root: Optional[TreeNode], targetSum: int) -> List[List[int]]:
		stack = []
		path = []
		ans, path_sum = [], 0
		cur_node = root

		while stack or cur_node:
			if cur_node:
				stack.append((cur_node, 1))
				path.append(cur_node.val)
				# print(path)
				path_sum += cur_node.val
				cur_node = cur_node.left
			else:
				node, flag = stack[-1]
				if flag == 1:
					cur_node = node
					stack[-1] = (node, 2)
					cur_node = node.right
				else:
					cur_node = stack.pop()[0]
					if not cur_node.left and not cur_node.right and path_sum == targetSum:
						ans.append(path.copy())
					path.pop()
					path_sum -= cur_node.val 
					cur_node = None
		return ans

root = [5,4,8,11,None,13,4,7,2,None,None,5,1]
targetSum = 22

root = constructBinaryTree(root)
solution = Solution()
solution.pathSum(root, targetSum)

[[5, 4, 11, 2], [5, 8, 4, 5]]

In [4]:
class Solution:
	# Prefix Sum + Backtracking
	def pathSum(self, root: Optional[TreeNode], targetSum: int) -> int:
		# record the number of nodes (start node counts) of the prefix sum.
		hashmap = {0: 1}

		def dfs(node: Optional[TreeNode], curSum: int) -> int:
			if not node: return 0
			curSum += node.val
			
			path_cnt = hashmap.get(curSum - targetSum, 0)
			hashmap[curSum] = hashmap.get(curSum, 0) + 1
			path_cnt += dfs(node.left, curSum) + dfs(node.right, curSum)
			
			# backtracking
			hashmap[curSum] -= 1
			return path_cnt
		
		return dfs(root, 0)
	
root = [10,5,-3,3,2,None,11,3,-2,None,1]
targetSum = 8
root = constructBinaryTree(root)

solution = Solution()
solution.pathSum(root, targetSum)

3

In [4]:
class Solution:
	def flatten(self, root: Optional[TreeNode]) -> None:
		if not root: return

		stack = [root]
		prev_node = None

		while stack:
			node = stack.pop()
			if prev_node:
				prev_node.left = None
				prev_node.right = node
			
			if node.right: stack.append(node.right)
			if node.left: stack.append(node.left)

			prev_node = node

root = [1,2,5,3,4,None,6]
root = constructBinaryTree(root)
solution = Solution()
solution.flatten(root)
convertBinaryTree2List(root)

[1, None, 2, None, 3, None, 4, None, 5, None, 6]

In [ ]:
# 108. Convert Sorted Array to Binary Search Tree

class Solution:
    """Given an integer array `nums` where the elements are sorted in ascending order, convert it to a height-balanced BST.
    """
    def sortedArrayToBST(self, nums: List[int]) -> Optional[TreeNode]:
        """
        Time Complexity: O(n)
        Space Complexity: O(logn)
        """
        def inOrderTraversal(left, right):
            if left > right: return None

            mid = (left + right) // 2

            root = TreeNode(nums[mid])
            root.left = inOrderTraversal(left, mid-1)
            root.right = inOrderTraversal(mid+1, right)
            
            return root
        
        return inOrderTraversal(0, len(nums)-1)
    
nums = [-10,-3,0,5,9]
solution = Solution()
root = solution.sortedArrayToBST(nums)
tree = convertBinaryTree2List(root)
tree

[0, -10, 5, None, -3, None, 9]

In [ ]:
# 109. Convert Sorted List to Binary Search Tree

class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

def constructList(head: List[int]) -> Optional[ListNode]:
    if not head: return None
    
    head_node = ListNode(head[0])
    cur_node = head_node

    for i in range(1, len(head)):
        node = ListNode(head[i])
        cur_node.next = node
        cur_node = node
    return head_node

def printList(head: Optional[ListNode]):
    node = head
    while node:
        print(node.val, end=" ")
        node = node.next
    print("")

class Solution:
    """
    1. A simple method is to convert the List to sorted array and the problem is convert to 109.
    2. fast-slow pointer. Time complexity: log(nlogn)
    """
    def sortedListToBST(self, head: Optional[ListNode]) -> Optional[TreeNode]:
        if not head: return None

        slow, fast = head, head
        pre_slow = None

        while (fast and fast.next):
            pre_slow = slow
            slow = slow.next
            fast = fast.next.next

        root = TreeNode(slow.val)

        if pre_slow:
            pre_slow.next = None
            root.left = self.sortedListToBST(head)
        
        root.right = self.sortedListToBST(slow.next)

        return root

head = [-10, -3, 0, 5, 9]
list_head = constructList(head)
printList(list_head)

solution = Solution()
tree = solution.sortedListToBST(list_head)
convertBinaryTree2List(tree)

-10 -3 0 5 9 


[0, -3, 9, -10, None, 5]

In [ ]:
# 173. Binary Search Tree Iterator

class BSTIterator:
    """Implement the `BSTIterator` class that represents an iterator over the in-order traversal of a BST.
    """
    def __init__(self, root: Optional[TreeNode]):
        """BSTIterator(TreeNode root) Initializes an object of the BSTIterator class. The root of the BST is given as part 
        of the constructor. The pointer should be initialized to a non-existent number smaller than any element in the BST.
        """
        self.stack = []
        self.pointer = root

        while self.pointer:
            self.stack.append(self.pointer)
            self.pointer = self.pointer.left
        
        min_val = self.stack[-1].val
        self.pointer = TreeNode(min_val - 1, None, None)

    def next(self) -> int:
        """int next() Moves the pointer to the right, then returns the number at the pointer.
        """
        if self.hasNext():
            self.pointer = self.pointer.right
            while self.pointer:
                self.stack.append(self.pointer)
                self.pointer = self.pointer.left
            self.pointer = self.stack.pop()
            return self.pointer.val

    def hasNext(self) -> int:
        """boolean hasNext() Returns true if there exists a number in the traversal to the right of the pointer, 
        otherwise returns false.
        """
        if self.stack or self.pointer.right:
            return True
        return False
    
root = [7,3,15,None,None,9,20]
root = constructBinaryTree(root)

obj = BSTIterator(root)
print(obj.next())
print(obj.next())
print(obj.hasNext())
print(obj.next())
print(obj.hasNext())
print(obj.next())
print(obj.hasNext())
print(obj.next())
print(obj.hasNext())

3
7
True
9
True
15
True
20
False


In [41]:
# 230. K-th Smallest Element in a BST

class Solution:
    def kthSmallest(self, root: Optional[TreeNode], k: int) -> int:
        stack, node = [], root

        while stack or node:
            if node:
                stack.append(node)
                node = node.left
            else:
                node = stack.pop()
                k -= 1
                if not k: return node.val
                node = node.right

root = [3,1,4,None,2]
k = 1
root = [5,3,6,2,4,None,None,1]
k = 3

root = constructBinaryTree(root)
solution = Solution()
ans = solution.kthSmallest(root, k)
ans

3

In [2]:
# 450. Delete Node in a BST

class Solution:
    def deleteNode(self, root: Optional[TreeNode], key: int) -> Optional[TreeNode]:
        if not root: return None

        if root.val > key:
            root.left = self.deleteNode(root.left, key)
        elif root.val < key:
            root.right = self.deleteNode(root.right, key)
        else:
            if not root.left: return root.right
            if not root.right: return root.left

            node = root.right

            while node.left:
                node = node.left
            
            node.left = root.left

            root = root.right
        
        return root

root = [5,3,6,2,4,None,7]
key = 3

root = constructBinaryTree(root)
solution = Solution()
ans = solution.deleteNode(root, key)
convertBinaryTree2List(ans)


[5, 4, 6, 2, None, None, 7]

In [24]:
# 530. Minimum Absolute Difference in BST

class Solution:
    def getMinimumDifference(self, root: Optional[TreeNode]) -> int:
        if not root: return None

        ans = None

        stack = []
        pre_node = None
        node = root

        while node or stack:
            if node:
                stack.append(node)
                node = node.left
            else:
                node = stack.pop()
                
                if pre_node:
                    diff = abs(node.val - pre_node.val)
                    if ans is None or diff < ans:
                        ans = diff
                if ans == 1: return 1
                pre_node = node
                node = node.right

        return ans

root = [4,2,6,1,3]
root = constructBinaryTree(root)

solution = Solution()
ans = solution.getMinimumDifference(root)
ans

1

In [38]:
# 538. Convert BST to Greater Tree

class Solution:
    def convertBST(self, root: Optional[TreeNode]) -> Optional[TreeNode]:
        if not root: return None

        stack = []
        node = root
        prefix_sum = 0

        while node or stack:
            if node:
                stack.append(node)
                node = node.right
            else:
                node = stack.pop()
                node.val = node.val + prefix_sum
                prefix_sum = node.val
                node = node.left

        return root
    
root = [4,1,6,0,2,5,7,None,None,None,3,None,None,None,8]
root = constructBinaryTree(root)

solution = Solution()
ans = solution.convertBST(root)
ans = convertBinaryTree2List(ans)
ans


[30, 36, 21, 36, 35, 26, 15, None, None, None, 33, None, None, None, 8]

In [39]:
# 653. Two Sum IV - Input is a BST

class Solution:
    def findTarget(self, root: Optional[TreeNode], k: int) -> bool:
        hashmap = set()

        def update(root: Optional[TreeNode], k: int):
            if not root: return False

            if k - root.val in hashmap:
                return True
            
            hashmap.add(root.val)

            return update(root.left, k) or update(root.right, k)

        return update(root, k)

root = [5,3,6,2,4,None,7]
k = 9

root = constructBinaryTree(root)
solution = Solution()
ans = solution.findTarget(root, k)
ans

True

In [2]:
# 669. Trim a Binary Search Tree

class Solution:
    def trimBST(self, root: Optional[TreeNode], low: int, high: int) -> Optional[TreeNode]:
        if not root: return None

        if root.val < low:
            return self.trimBST(root.right, low, high)
        if root.val > high:
            return self.trimBST(root.left, low, high)
        
        root.left = self.trimBST(root.left, low, high)
        root.right = self.trimBST(root.right, low, high)

        return root

root = [1, 0, 2]
low = 1
high = 2

root = [3, 0, 4, None, 2, None, None, 1]
low = 1
high = 3
root = constructBinaryTree(root)

solution = Solution()
ans = solution.trimBST(root, low, high)
convertBinaryTree2List(ans)

[3, 2, None, 1]

In [4]:
# 700. Search in a Binary Search Tree

class Solution:
    def searchBST(self, root: Optional[TreeNode], val: int) -> Optional[TreeNode]:
        while root and root.val != val:
            if root.val < val:
                root = root.right
            elif root.val > val:
                root = root.left
        
        return root

root = [4,2,7,1,3]
val = 2

root = constructBinaryTree(root)
solution = Solution()
ans = solution.searchBST(root, val)
convertBinaryTree2List(ans)

[2, 1, 3]

In [7]:
# 701. Insert into a Binary Search Tree

class Solution:
    def insertIntoBST(self, root: Optional[TreeNode], val: int) -> Optional[TreeNode]:
        if not root: return TreeNode(val, None, None)
        node = root
        while node:
            if node.val < val:
                if node.right:
                    node = node.right
                else:
                    node.right = TreeNode(val, None, None)
                    return root
            elif node.val > val:
                if node.left:
                    node = node.left
                else:
                    node.left = TreeNode(val, None, None)
                    return root

    def insertIntoBST(self, root: Optional[TreeNode], val: int) -> Optional[TreeNode]:

        if not root: return TreeNode(val, None, None)

        if root.val > val:
            root.left = self.insertIntoBST(root.left, val)
        else:
            root.right = self.insertIntoBST(root.right, val)
        
        return root
            

root = [4,2,7,1,3]
val = 5

root = constructBinaryTree(root)
solution = Solution()
ans = solution.insertIntoBST(root, val)
convertBinaryTree2List(ans)

[4, 2, 7, 1, 3, 5]

In [12]:
# 783. Minimum Distance Between BST Nodes

class Solution:
    def minDiffInBST(self, root: Optional[TreeNode]) -> int:
        stack = []
        pre_node = None
        node = root
        ans = None

        while node or stack:
            if node:
                stack.append(node)
                node = node.left
            else:
                node = stack.pop()
                
                if pre_node:
                    diff = node.val - pre_node.val
                    if diff == 1:
                        return 1
                    else:
                        if not ans or ans > diff:
                            ans = diff

                pre_node = node
                node = node.right
        return ans

root = [4,2,6,1,3]
root = constructBinaryTree(root)
solution = Solution()
ans = solution.minDiffInBST(root)
ans

1

In [29]:
# 897. Increasing Order Search Tree

class Solution:
    def increasingBST(self, root: Optional[TreeNode]) -> Optional[TreeNode]:
        stack = []
        new_root = None
        pre_node = None
        node = root

        while node or stack:
            if node:
                stack.append(node)
                node = node.left
            else:
                node = stack.pop()
                if not new_root:
                    new_root = node
                else:
                    pre_node.left = None
                    pre_node.right = node
                pre_node = node
                node = node.right
        pre_node.left = None
        pre_node.right = None
        return new_root


root = [5,3,6,2,4,None,8,1,None,None,None,7,9]
root = [5,1,7]
root = [2,1,4,None,None,3]
root = constructBinaryTree(root)
solution = Solution()
new_root = solution.increasingBST(root)
convertBinaryTree2List(new_root)
                

[1, None, 2, None, 3, None, 4]

In [30]:
class Solution:
    def rangeSumBST(self, root: Optional[TreeNode], low: int, high: int) -> int:
        def update_sum(root, low, high):
            if not root: return 0

            if root.val < low:
                return update_sum(root.right, low, high)
            if root.val > high:
                return update_sum(root.left, low, high)
            return root.val + update_sum(root.left, low, high) + update_sum(root.right, low, high)
        return update_sum(root, low, high)

root = [10,5,15,3,7,13,18,1,None,6]
low = 6
high = 10

root = constructBinaryTree(root)
solution = Solution()
solution.rangeSumBST(root, low, high)

23

In [ ]:
# 1008. Construct Binary Search Tree from Preorder Traversal

class Solution:
    def bstFromPreorder(self, preorder: List[int]) -> Optional[TreeNode]:
        stack = []
        root = TreeNode(preorder[0])
        node = root
        
        for val in preorder[1:]:
            if val < node.val:
                node.left = TreeNode(val)
                stack.append(node)
                node = node.left
            else:
                while stack:
                    if stack[-1].val > val:
                        break
                    node = stack.pop()
                node.right = TreeNode(val)
                node = node.right
                stack.append(node)
        return root
                    
preorder = [8,5,1,7,10,12]
solution = Solution()
root = solution.bstFromPreorder(preorder)
convertBinaryTree2List(root)

[8, 5, 10, 1, 7, None, 12]